In [44]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import math
import screeninfo
import freenect

In [45]:
%matplotlib
def plot(string='',*images):
    ncols=2
    nrows=int(math.ceil(len(images)/2.0))
    fig, ax = plt.subplots(nrows, ncols,gridspec_kw = {'wspace':0, 'hspace':0},facecolor='.5')
    plt.axis('off')
    for i in range(0,len(images)):
        plt.subplot(nrows,ncols,i+1),plt.imshow(images[i],cmap='binary_r')
        plt.text(0,0,string+'\n')
        plt.axis('off')
    plt.show()

Using matplotlib backend: TkAgg


In [46]:
def make_rect_points(cv_rw,cv_cl):
    h=150
    w=200
    return [(cv_cl,cv_rw),(cv_cl+w,cv_rw),(cv_cl,cv_rw+h),(cv_cl+w,cv_rw+h)]

In [47]:
def get_frame():
    array,_ = freenect.sync_get_video()
    array = cv2.cvtColor(array,cv2.COLOR_RGB2GRAY)
    return array

In [48]:
def find_homo(src,dst):
    h, status = cv2.findHomography(src,dst)
    return h

In [49]:
def find_dest(in_points,h):
    points=[]
    for point in in_points:
        point=list(point)
        point.append(1)
        homo_points=np.matmul(h,point)
        points.append((int(homo_points[0]/homo_points[2]),int(homo_points[1]/homo_points[2])))
    return points

In [50]:
def make_project_points(rows=768,colms=1366,pnts=(400,700)):
    screen_id = 1
    is_color = False

    # get the size of the screen
    screen = screeninfo.get_monitors()[screen_id]
    colms, rows = screen.width, screen.height

    # create image
    if is_color:
        img = np.zeros((rows,colms,3), np.uint8)
        pts_src=make_rect_points(pnts[0],pnts[1])
        for pts in pts_src: 
            cv2.circle(img,(pts[0],pts[1]),20, (255,255,255), -1)
    else:
        img = np.zeros((rows,colms,3), np.uint8)
        pts_src=make_rect_points(pnts[0],pnts[1])
        for pts in pts_src: 
            cv2.circle(img,(pts[0],pts[1]),20, (255,255,255), -1)
    #cv2.circle(img,(401, 519),10, (255,255,255), -1)
    window_name = 'projector'
    cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
    cv2.moveWindow(window_name, screen.x - 1, screen.y - 1)
    cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN,
                          cv2.WINDOW_FULLSCREEN)
    cv2.imshow(window_name, img)
    cv2.waitKey(0)
    
    return [img, pts_src]


In [51]:
def find_cam_points():
    img= get_frame()
    cv2.imwrite("./captured.jpg",img)
    plot('grabbed',img  )
    ret,th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    plot("threshold",th1)
    mask = cv2.erode(th1, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=3)
    plot("mask",mask)
    im, contours, hierarchy = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_cnts = sorted(contours,key= lambda cnt: int(cv2.minEnclosingCircle(cnt)[1]), reverse=True)[:4]
    sum_arry=[]
    diff_arry=[]
    ponts=[]#debug
    for cnt in sorted_cnts:
        (x,y),radius = cv2.minEnclosingCircle(cnt)
        sum_arry.append(x+y)
        diff_arry.append(x-y)
        ponts.append((int(x),int(y)))
    top_left = np.argmin(sum_arry)
    bot_right=np.argmax(sum_arry)
    top_right=np.argmax(diff_arry)
    bot_left=np.argmin(diff_arry)
    return [ponts[top_left],ponts[top_right],ponts[bot_left],ponts[bot_right]]


In [52]:
im_po=make_project_points()

src=find_cam_points()



dst=make_rect_points(400,700)

h=find_homo(np.array(src),np.array(dst))


cv2.destroyAllWindows()

In [53]:
dictionary = {'homo':h}
np.save('homography.npy', dictionary)